Requires:
```
conda install -c conda-forge ipympl widgetsnbextension
```

proj.xy2ang() + https://matplotlib.org/3.1.1/users/event_handling.html

In [1]:
import os
import sys
_path = os.path.abspath('../')
if _path not in sys.path:
    sys.path.append(_path)

import astropy.coordinates as coord
import astropy.table as at
from astropy.io import fits
import astropy.units as u
import gala.coordinates as gc

%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
# %matplotlib qt5
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
from scipy.ndimage import gaussian_filter

import numpy as np
import healpy as hp
from tqdm import tqdm, trange

from stream_helpers import make_anim_frames, get_data, rots, zyx_euler_from_endpoints, StreamRidgeBuilder

In [2]:
cube, dms, footprint_mask = get_data(bass_file='../data/BASS_iso_hpxcube_z0.0001_a13.5_gmax23.fits.gz', 
                                     decals_file='../data/DECaLS_iso_hpxcube_z0.0001_a13.5_gmax23.fits.gz')
npix, nslice = cube.shape
nside = hp.npix2nside(npix)
dm_dist = coord.Distance(distmod=dms)

In [3]:
slice_idx = slice(23, 26)
sub_cube = cube[:, slice_idx].sum(axis=1) / (slice_idx.stop - slice_idx.start)
print(dms[slice_idx][0], dms[slice_idx][-1])

16.29999999999999 16.499999999999993


In [6]:
rot = zyx_euler_from_endpoints(23.9, -49.9, 26.61, -44.7) # Phoenix
# rot = zyx_euler_from_endpoints(19.5, -26.6, 31, -33.) # ATLAS
proj = hp.projector.GnomonicProj(xsize=1024, ysize=512, 
                                 rot=rot, reso=4.)

func = lambda x, y, z: hp.vec2pix(nside, x, y, z)
img = proj.projmap(sub_cube, func)
img = gaussian_filter(img, 1.5)

In [7]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

_mask = (img != 0) & np.isfinite(img)
ax.imshow(img, 
          origin='bottom', extent=proj.get_extent(),
          vmin=np.nanpercentile(img[_mask], 1),
          vmax=np.nanpercentile(img[_mask], 95), 
          cmap='Greys')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
fig.tight_layout()

gc_builder = StreamRidgeBuilder(ax, proj)
            
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [85]:
zyx_euler_from_endpoints(23.903172489801012, -49.93874392389452, 26.613474341221124, -44.72548640646065)

[25.3252684156842, -47.34008274830022, 73.30501725767499]

In [78]:
[[19.563572253712774, -55.80817703695607], [21.729323031565805, -52.75856518655871], [23.346253323894064, -49.97912456789504], [25.228400824779282, -47.52990507690572], [27.108652383715288, -44.377437782282186], [29.36150435106662, -38.20690425357492], [30.91081441221493, -33.6492882922184]]

[[23.610543624214518, -50.00904115781739],
 [24.90836300640222, -48.44208272133352],
 [25.707386306512177, -46.906041032575985],
 [26.527340913681236, -44.91889034642702],
 [27.247458090744537, -43.196624849055354],
 [33.8125266088898, -41.01644043006024],
 [33.8125266088898, -41.01644043006024]]

In [136]:
fr = gc.GreatCircleICRSFrame(pole=coord.SkyCoord(142.1977106, 53.25641986, unit='deg'),
                             ra0=232.0451099160*u.deg)

In [137]:
derp = fr.realize_frame(coord.UnitSphericalRepresentation(0*u.deg, 0*u.deg)).transform_to(coord.ICRS)
derp

<ICRS Coordinate: (ra, dec) in deg
    (232.04510992, -0.11392542)>

In [138]:
R = gc.greatcircle.reference_to_greatcircle(coord.ICRS, fr)
herp = np.arctan2(R[2,1] / np.cos(derp.dec), 
                  R[2,2] / np.cos(derp.dec)).to_value(u.deg)
herp

24.588643649856685

In [143]:
# rot = [derp.ra.degree, derp.dec.degree, herp]
rot = [27.47097460294872, -4.868245033819535, 30.08377796375123]

In [144]:
proj = hp.projector.MollweideProj(xsize=1024, 
                                  rot=rot)
img = proj.projmap(map_, func) 

fig, ax = plt.subplots(1, 1, figsize=(10, 5))

_mask = (img != 0) & np.isfinite(img)
ax.imshow(img, origin='bottom', extent=proj.get_extent(),
          vmin=np.nanpercentile(img[_mask], 1),
          vmax=np.nanpercentile(img[_mask], 95))
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [9]:
class GreatCircleBuilder:

    def __init__(self, ax, proj):
        self.ax = ax
        self.reset()
        
        self.cids = {}
        self.cids['mouse'] = self.ax.figure.canvas.mpl_connect('button_press_event',
                                                               self)
        self.cids['key'] = fig.canvas.mpl_connect('key_press_event', self)


    def reset(self):
        self.line = None
        self.xs = list()
        self.ys = list()

    def __call__(self, event):
        if event.inaxes != self.ax:
            return

        self.xs.append(event.xdata)
        self.ys.append(event.ydata)

        if self.line is None:
            self.line, = self.ax.plot(self.xs, self.ys, color='tab:red')
        else:
            self.line.set_data(self.xs, self.ys)

        self.line.figure.canvas.draw()
        
        print(event.key)

        if len(self.xs) >= 2:
            endpoints = [proj.xy2ang(x, y, lonlat=True) 
                         for x, y in zip(self.xs, self.ys)] * u.deg
            c1 = coord.SkyCoord(endpoints[0, 0], endpoints[0, 1])
            c2 = coord.SkyCoord(endpoints[1, 0], endpoints[1, 1])
            fr = gc.GreatCircleICRSFrame.from_endpoints(c1, c2)
            origin = fr.realize_frame(coord.UnitSphericalRepresentation(0*u.deg, 0*u.deg))
            print(fr)
            gc_icrs = origin.transform_to(coord.ICRS)
            R = gc.greatcircle.reference_to_greatcircle(coord.ICRS, fr)
            psi = -np.degrees(np.arctan2(R[2,1], R[2,2]))
            print([gc_icrs.ra.degree, gc_icrs.dec.degree, psi])
            
            self.line.set_data([], [])
            self.reset()